In [1]:
import numpy as np
import sys
from time import time

In [3]:
class Node:
    def __init__(self, parent, state, pcost, hcost):
        """
        Initializes a new instance of the Node class.
        
        Args:
            parent (Node): The parent node of the current node.
            state: The state associated with the current node.
            pcost (float): The path cost associated with the current node.
            hcost (float): The heuristic cost associated with the current node.
        """
        self.parent = parent
        self.state = state
        self.pcost = pcost
        self.hcost = hcost
        self.cost = pcost + hcost # The total cost of the node is the sum of the path cost and heuristic cost.
    
    def __hash__(self):
        """
        Computes a hash value for the current node.
        """
        return hash(''.join(self.state.flatten())) # We flatten the state matrix and join all the elements to form a string, which is then hashed.
    
    def __str__(self):
        """
        Returns a string representation of the current node.
        """
        return str(self.state)
    
    def __eq__(self, other):
        """
        Compares the current node with another node for equality.
        
        Args:
            other (Node): The other node to compare with.
        """
        return hash(''.join(self.state.flatten())) == hash(''.join(other.state.flatten())) # We compare the hash values of the flattened state matrices.
    
    def __ne__(self, other):
        """
        Compares the current node with another node for inequality.
        
        Args:
            other (Node): The other node to compare with.
        """
        return hash(''.join(self.state.flatten())) != hash(''.join(other.state.flatten())) # We compare the hash values of the flattened state matrices.


In [4]:
class PriorityQueue():
    
    def __init__(self):
        """
        Initializes a new instance of the PriorityQueue class.
        """
        self.queue = []
        
    def push(self, node):
        """
        Adds a node to the priority queue.
        
        Args:
            node (Node): The node to add to the priority queue.
        """
        self.queue.append(node)
    
    def pop(self):
        """
        Removes and returns the node with the lowest cost from the priority queue.
        """
        next_state = None
        state_cost = 10**18 # Initialize state_cost to a high value.
        index = -1
        
        for i in range(len(self.queue)):
            if self.queue[i].cost < state_cost:
                state_cost = self.queue[i].cost # Update state_cost to the cost of the current node if it's lower than the current state_cost.
                index = i # Store the index of the node with the lowest cost so far.
        
        return self.queue.pop(index) # Remove and return the node with the lowest cost.
    
    def is_empty(self):
        """
        Checks if the priority queue is empty.
        """
        return len(self.queue) == 0
    
    def __str__(self):
        """
        Returns a string representation of the priority queue.
        """
        l = []
        for i in self.queue:
            l.append(i.state)
        
        return str(l)
    
    def __len__(self):
        """
        Returns the number of nodes in the priority queue.
        """
        return len(self.queue)


In [5]:
import numpy as np

class Environment():
    
    def __init__(self, depth=None, goal_state=None):
        # List of possible actions: 1 - Up, 2 - Down, 3 - Right, 4 - Left
        self.actions = [1, 2, 3, 4] 
        self.goal_state = goal_state # The state we are trying to reach
        self.depth = depth # Maximum depth to generate the start state
        self.start_state = self.generate_start_state() # Generate a start state
    
    def generate_start_state(self):
        # This function generates a random start state with the given depth
        
        # Start with the goal state
        past_state = self.goal_state
        i = 0
        
        while i != self.depth:
            new_states = self.get_next_states(past_state) # Get all possible next states
            choice = np.random.randint(low=0, high=len(new_states)) # Choose a random next state
            
            if np.array_equal(new_states[choice], past_state):
                # If the chosen state is the same as the previous state, continue
                continue
            
            past_state = new_states[choice] # Set the current state to the chosen next state
            i += 1
            
        return past_state # Return the generated start state
    
    def get_start_state(self):
        return self.start_state # Return the generated start state
    
    def get_goal_state(self):
        return self.goal_state # Return the goal state
    
    def get_next_states(self, state):
        # This function returns all possible next states for a given state
        
        space = (0, 0)
        for i in range(3):
            for j in range(3):
                if state[i, j] == '_':
                    space = (i, j)
                    break
        
        new_states = [] # List to store the possible next states
        
        if space[0] > 0: # Move Up
            new_state = np.copy(state)
            
            val = new_state[space[0], space[1]]
            new_state[space[0], space[1]] = new_state[space[0]-1, space[1]]
            new_state[space[0]-1, space[1]] = val
            
            new_states.append(new_state)
            
        if space[0] < 2: # Move Down
            new_state = np.copy(state)
            
            val = new_state[space[0], space[1]]
            new_state[space[0], space[1]] = new_state[space[0]+1, space[1]]
            new_state[space[0]+1, space[1]] = val
            
            new_states.append(new_state)
        
        if space[1] < 2: # Move Right
            new_state = np.copy(state)
            
            val = new_state[space[0], space[1]]
            new_state[space[0], space[1]] = new_state[space[0], space[1]+1]
            new_state[space[0], space[1]+1] = val
            
            new_states.append(new_state)
            
        if space[1] > 0: # Move Left
            new_state = np.copy(state)
            
            val = new_state[space[0], space[1]]
            new_state[space[0], space[1]] = new_state[space[0], space[1]-1]
            new_state[space[0], space[1]-1] = val
            
            new_states.append(new_state)
        
        return new_states # Return the possible next states
    


In [7]:
class Agent:
    
    def __init__(self, env, heuristic):
        self.frontier = PriorityQueue() #Initialize priority queue for nodes
        self.explored = dict() #Dictionary to keep track of explored nodes
        self.start_state = env.get_start_state() #Get start state from the environment
        self.goal_state = env.get_goal_state() #Get goal state from the environment
        self.env = env #Assign environment object to the agent
        self.goal_node = None #Initialize goal node to None
        self.heuristic = heuristic #Assign heuristic function to the agent
    
    def run(self):
        init_node = Node(parent = None, state = self.start_state, pcost = 0, hcost=0) #Create initial node with start state, zero path cost and heuristic cost
        self.frontier.push(init_node) #Add initial node to the frontier
        steps = 0 #Initialize steps counter to zero
        while not self.frontier.is_empty(): #Loop until the frontier is empty

            curr_node = self.frontier.pop() #Remove and return the node with the lowest cost
            next_states = self.env.get_next_states(curr_node.state) #Get next possible states from current node's state

            if hash(curr_node) in self.explored: #If current node is already explored, skip it
                continue

            self.explored[hash(curr_node)] = curr_node #Add current node to the explored dictionary with its hash as key

            if self.env.reached_goal(curr_node.state): #If current node's state is the goal state, set the goal node and break out of the loop
                self.goal_node = curr_node
                break
            goal_state = self.env.get_goal_state()

            for state in next_states: #Loop over the next possible states
                hcost = self.heuristic(state, goal_state) #Calculate heuristic cost for the state
                node = Node(parent=curr_node, state=state, pcost=curr_node.pcost+1, hcost=hcost) #Create a new node with the state, path cost and heuristic cost
                self.frontier.push(node) #Add the new node to the frontier
            steps += 1 #Increment the step counter
        
        return steps, self.soln_depth() #Return the number of steps taken and the solution depth

    def soln_depth(self):
        node = self.goal_node #Start from the goal node
        count = 0 #Initialize counter to zero
        while node is not None: #Loop until there are no more nodes
            node = node.parent #Move to the parent node
            count+=1 #Increment the counter
        
        return count #Return the solution depth
    
    def print_nodes(self):
        node = self.goal_node #Start from the goal node
        l = []
        while node is not None: #Loop until there are no more nodes
            l.append(node) #Add the node to the list
            node = node.parent #Move to the parent node

        step = 1 #Initialize step counter to one
        for node in l[::-1]: #Loop over the nodes in reverse order
            print("Step: ",step) #Print the step number
            print(node) #Print the node
            step+=1 #Increment the step counter
    
    def get_memory(self):
        mem = len(self.frontier)*56 + len(self.explored)*56 #Calculate the memory used by the frontier and explored dictionary
        return mem #Return the memory used


Heuristic 0

In [8]:
def heuristic0(curr_state, goal_state):
    return 0

Heuristic 1

In [9]:
def heuristic1(curr_state, goal_state):
    """
    This function computes a heuristic value for the given current state of the puzzle
    based on the number of misplaced tiles in the current state compared to the goal state.

    Parameters:
        curr_state (numpy.ndarray): The current state of the puzzle.
        goal_state (numpy.ndarray): The goal state of the puzzle.

    Returns:
        count (int): The number of misplaced tiles in the current state.
    """

    # Initialize a counter to keep track of the number of misplaced tiles.
    count = 0
    
    # Loop through each cell in the puzzle grid.
    for i in range(3):
        for j in range(3):
            # If the value of the current cell in the current state is not equal to the value of the
            # corresponding cell in the goal state, then increment the counter.
            if curr_state[i, j] != goal_state[i, j]:
                count += 1
    
    # Return the final count of misplaced tiles.
    return count


Heuristic 2

In [10]:
import numpy as np

def heuristic2(curr_state, goal_state):
    """
    This function computes a heuristic value for the given current state of the puzzle
    based on the Manhattan distance between each tile in the current state and its
    corresponding tile in the goal state.

    Parameters:
        curr_state (numpy.ndarray): The current state of the puzzle.
        goal_state (numpy.ndarray): The goal state of the puzzle.

    Returns:
        dist (int): The sum of the Manhattan distances between each tile in the current
                    state and its corresponding tile in the goal state.
    """

    # Initialize a variable to keep track of the total Manhattan distance.
    dist = 0

    # Loop through each cell in the puzzle grid.
    for i in range(3):
        for j in range(3):
            # Get the value of the current cell in the current state.
            ele = curr_state[i, j]

            # Find the indices of the same value in the goal state.
            goal_i, goal_j = np.where(goal_state == ele)

            # Calculate the Manhattan distance between the current cell and the corresponding cell in the goal state.
            d = abs(goal_i[0] - i) + abs(goal_j[0] - j)

            # Add the distance to the total distance.
            dist += d
    
    # Return the final sum of Manhattan distances.
    return dist


In [11]:
import numpy as np

# Define the depth of the search tree.
depth = 500

# Define the goal state of the puzzle.
goal_state = np.array([[1, 2, 3], [8, '_', 4], [7, 6, 5]])

# Create a new environment with the specified depth and goal state.
env = Environment(depth, goal_state)

# Print the start state of the puzzle.
print("Start State:")
print(env.get_start_state())

# Print the goal state of the puzzle.
print("Goal State:")
print(goal_state)


Start State:
[['8' '3' '1']
 ['6' '4' '7']
 ['2' '5' '_']]
Goal State:
[['1' '2' '3']
 ['8' '_' '4']
 ['7' '6' '5']]


In [13]:
agent = Agent(env = env, heuristic = heuristic2)


In [16]:
depths = np.arange(0,501,50)
goal_state = np.array([[1,2,3], [8,'_',4], [7,6,5]])
times_taken = {}
mems = {}
for depth in depths:
    
    time_taken = 0
    mem = 0
    for i in range(50):
        env = Environment(depth=depth, goal_state=goal_state)
        agent = Agent(env = env, heuristic = heuristic2)
        start_time = time()
        end_time = time()
        time_taken+=end_time - start_time
        mem+=agent.get_memory()
    
    time_taken/=50
    mem = mem/50
    times_taken[depth] = time_taken
    mems[depth] = mem
    print(depth, time_taken, mem)

    

0 1.811981201171875e-07 0.0
50 5.769729614257812e-07 0.0
100 3.6716461181640626e-07 0.0
150 2.86102294921875e-07 0.0
200 3.3855438232421873e-07 0.0
250 3.290176391601562e-07 0.0
300 3.5762786865234375e-07 0.0
350 3.337860107421875e-07 0.0
400 3.1948089599609377e-07 0.0
450 4.38690185546875e-07 0.0
500 3.9577484130859373e-07 0.0
